In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("credit_risk_dataset.csv")
df = df.dropna()

x = df[['person_age', 'person_income']].values
y = df[['loan_status']].values

x = torch.tensor(x,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.float32)

dataset = data_utils.TensorDataset(x, y)
train_size = int(len(dataset)*0.8)
test_size = len(dataset) - train_size
train_set, test_set = data_utils.random_split(dataset, [train_size, test_size])

train_loader = data_utils.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = data_utils.DataLoader(test_set, shuffle=False)

In [3]:
class SimpleNet(nn.Module):
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(input_size, 2)
        self.layer2 = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.sigmoid(out)
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

In [4]:
# Hyperparameters
learning_rate = 0.0001
num_epochs = 50
weight_decay = 0.005
momentum = 0.9

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD

# Model definition
input_size = 2
model = SimpleNet(input_size)

# Optimizer
optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

In [5]:
%%time
for epoch in range(num_epochs):
    for i, sample in enumerate(train_loader):
        # Forward
        x, y = sample
        output = model(x)
        
        # loss
        loss = criterion(output, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))         

CPU times: user 15.1 s, sys: 8.58 ms, total: 15.1 s
Wall time: 15.2 s


In [6]:
 with torch.no_grad():
    acc = 0
    total = len(test_loader)
    for x, y in test_loader:
        y_pred = model(x)
        acc += (y_pred.round() == y).float().mean()
    
    print('Accuracy of the network on the {} test samples: {:.2f} %'.format(total, 100 * acc / total)) 

Accuracy of the network on the 5728 test samples: 78.75 %


In [7]:
input_sample, _ = next(iter(train_loader))
input_names = ['person_age', 'person_income', 'person_emp_length']
output_names = ['loan_status']
torch.onnx.export(model, input_sample, "credit_risk.onnx", input_names=input_names, output_names=output_names)

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [8]:
from visualdl.server import app
visualdl.server.app.run(model="credit_risk.onnx",
                        host="gr13sc02",
                        port=8080,
                        cache_timeout=20,
                        language=None,
                        public_path=None,
                        api_only=False,
                        open_browser=False)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'Mapping' from 'collections' (/opt/conda/lib/python3.10/collections/__init__.py)